# Задача
Решаем задачу оптимизации $f(x)\to\underset{\mathbb{R}^n}{\min}$, где $f(x)=\frac{1}{2}x^TAx-b^Tx$, причем матрица $A$ симметричная и положительно определенная

# Библиотеки для работы с матрицами

In [2]:
import numpy as np
from numpy import linalg

# Создание случайной матрицы

$$A=S^TDS$$
S получается ортогонализацией Грамма-Шмидта из случайной матрицы, D задается явно ниже

In [169]:
# Размерность пространства
n=200
# Случайная матрица
S0=np.random.randn(n,n)

In [170]:
# Ортогонализация Грамма-Шмидта
S=np.copy(S0)
for i in range(n):
    S[i] = np.copy(S0[i])
    for j in range(i):
        S[i] -= S[j] * np.dot(S[i], S[j]) / np.dot(S[j], S[j])
    S[i] /= np.sqrt(np.dot(S[i], S[i]))

In [171]:
# Проверяем ортогональность
np.allclose(np.dot(S.T,S), np.eye(n))

True

In [179]:
# Собственные числа матрицы A
D=np.diag(np.array(range(n)) + 1)

In [180]:
print D

[[  1   0   0 ...,   0   0   0]
 [  0   2   0 ...,   0   0   0]
 [  0   0   3 ...,   0   0   0]
 ..., 
 [  0   0   0 ..., 198   0   0]
 [  0   0   0 ...,   0 199   0]
 [  0   0   0 ...,   0   0 200]]


In [181]:
A=np.dot(S.T, np.dot(D, S))

In [182]:
# Вектор b
b=np.random.rand(n)

In [183]:
# Градиент функции
def grad(A, b, x):
    return(np.dot(A, x) - b)

In [184]:
# Функция
def func(A, b, x):
    return(0.5 * np.dot(x.T, np.dot(A, x))-np.dot(b, x))

## Метод 1. Градиентный спуск с постоянным шагом

$$s_k=-\nabla f(x_k)$$
$$x_{k+1}=x_k+\alpha_ks_k$$

In [185]:
x=list()
x.append(np.zeros(n))
i = 1
a=0.005

x_norm=1e-6
maxiter=10000

x_min = None
f_min = None
i_min = None

while True:
    x.append(0)
    x[i] = x[i-1] - a * grad(A, b, x[i - 1])
    #print x[i], func(A, b, x[i])
    
    f_now = func(A, b, x[i])
    
    if f_min == None or f_now < f_min :
        f_min = f_now
        x_min = x[i]
        i_min = i
    
    if(linalg.norm(x[i]-x[i - 1]) < x_norm):
        break
    
    if i >= maxiter:
        break
        
    i += 1

np.set_printoptions(precision=3)
print "i =", i
print "i_min =", i_min
print "func =", f_min
print "x =", x_min
print "||grad|| = ||Ax-b|| =", linalg.norm(np.dot(A, x_min) - b)

i = 1148
i_min = 1148
func = -1.29838583644
x = [-0.025  0.06   0.084  0.05   0.011  0.097  0.033  0.09   0.077 -0.02
  0.046  0.031  0.011  0.026  0.064 -0.152  0.026  0.052  0.052  0.042
 -0.004  0.009  0.016 -0.016  0.04   0.001  0.018  0.05  -0.031  0.052
  0.051 -0.005 -0.007 -0.003  0.01   0.068  0.104  0.018  0.044  0.086
 -0.019 -0.004  0.051  0.058  0.063  0.048  0.047  0.109  0.045 -0.048
 -0.048  0.057  0.011  0.033 -0.014  0.082  0.01  -0.012 -0.002  0.048
  0.013 -0.003  0.041  0.033  0.108 -0.079 -0.045  0.     0.046  0.032
  0.098 -0.025  0.007  0.044 -0.041  0.054 -0.03   0.021  0.136  0.002
 -0.033  0.012  0.03   0.037  0.035  0.085 -0.049  0.134 -0.049 -0.044
  0.046  0.042 -0.041  0.01  -0.036  0.041  0.059  0.078  0.082  0.041
  0.002 -0.004  0.013  0.035  0.078  0.065  0.181  0.007  0.066  0.028
  0.013  0.04   0.056 -0.05  -0.107  0.011 -0.041 -0.049  0.034  0.074
  0.015  0.019 -0.025  0.022  0.004 -0.03   0.002  0.076  0.128  0.021
 -0.041  0.003 -0.081  0.04   

## Метод 2. Градиентный спуск, выбор шага по правилу Армихо, остановка по $||\nabla f||$

In [188]:
# Массив с точками
x=list()
x.append(np.zeros(n))
i = 1

# Выбор шага по Армихо, параметры
a0 = 1
theta = 0.6
eps = 0.6

# Результат
x_min = None
f_min = None
i_min = None

# Условия остановки
grad_norm=1e-4
maxiter=10000

while True:
    x.append(0)
    
    # Выбор шага по Армихо
    a = a0
    grad_norm_2 = np.linalg.norm(grad(A, b, x[i - 1])) ** 2
    while True:
        x_new = x[i - 1] - a * grad(A, b, x[i - 1])
        if func(A, b, x_new) - func(A, b, x[i - 1]) + eps * a * grad_norm_2 < 0:
            break
        a *= theta
    
    #print a
    
    # Градиентный шаг
    x[i] = x[i - 1] - a * grad(A, b, x[i - 1])
    #print x[i], func(A, b, x[i])
    
    # Результат
    x_min = x[i]
    i_min = i
    f_min = func(A, b, x_min)
    
    # Условие останова
    if np.linalg.norm(grad(A, b, x[i])) < grad_norm:
        break
    
    if i >= maxiter:
        break
        
    i += 1

np.set_printoptions(precision=3)
print "i =", i
print "i_min =", i_min
print "func =", f_min
print "x =", x_min
print "||grad|| = ||Ax-b|| =", linalg.norm(np.dot(A, x_min) - b)

i = 358
i_min = 358
func = -1.29838585232
x = [-0.025  0.06   0.084  0.05   0.011  0.097  0.033  0.09   0.077 -0.02
  0.046  0.031  0.011  0.026  0.064 -0.152  0.026  0.052  0.052  0.042
 -0.004  0.009  0.016 -0.016  0.04   0.001  0.018  0.05  -0.031  0.052
  0.051 -0.005 -0.007 -0.003  0.01   0.068  0.104  0.018  0.044  0.086
 -0.019 -0.004  0.051  0.058  0.063  0.048  0.047  0.109  0.045 -0.048
 -0.048  0.057  0.011  0.033 -0.014  0.082  0.01  -0.012 -0.002  0.048
  0.013 -0.003  0.041  0.033  0.108 -0.079 -0.045  0.     0.046  0.032
  0.098 -0.025  0.007  0.044 -0.041  0.054 -0.03   0.021  0.136  0.002
 -0.033  0.012  0.03   0.037  0.035  0.085 -0.049  0.134 -0.049 -0.044
  0.046  0.042 -0.041  0.01  -0.036  0.041  0.059  0.078  0.082  0.041
  0.002 -0.004  0.013  0.035  0.078  0.065  0.181  0.007  0.066  0.028
  0.013  0.04   0.056 -0.05  -0.107  0.011 -0.041 -0.049  0.034  0.074
  0.015  0.019 -0.025  0.022  0.004 -0.03   0.002  0.076  0.128  0.021
 -0.041  0.003 -0.081  0.04   0.

## Метод 3. Градиентный спуск, выбор шага по правилу одномерной минимизации, остановка по $||\nabla f||$

In [190]:
# Массив с точками
x=list()
x.append(np.zeros(n))
i = 1

# С какого значения начинать поиск alpha?
a0 = 1
# Какая точность alpha нужна?
a_eps = 0.01

# Результат
x_min = None
f_min = None
i_min = None

# Условие останова
grad_norm=1e-4
maxiter=10000

# Функция одной переменной, которую будем минимизировать
def func1(x_, alpha):
    return(func(A, b, x_ - alpha * grad(A, b, x_)))

while True:
    x.append(0)
    
    # Выбор шага: одномерная минимизация,
    # Метод деления отрезка
    
    # Начальные границы отрезка
    a_a = 0.
    a_b = 1. * a0
    
    # Увеличиваем границу, пока функция не начнет увеличиваться
    while True:
        if func1(x[i - 1], a_b) > func1(x[i - 1], 0):
            break
        a_b *= 2
    
    #print "amax=", a_b
    
    # Ищем минимум функции одной переменной func1(x[i-1], a) по a
    while abs(a_a - a_b) >= a_eps:
        a_c = (a_a + a_b) / 2
        a_y = (a_a + a_c) / 2
        a_z = (a_y + a_b) / 2
        
        if func1(x[i - 1], a_y) <= func1(x[i - 1], a_c):
            a_b = a_c
            #a_c = a_y
        elif func1(x[i - 1], a_c) <= func1(x[i - 1], a_z):
            a_a = a_y
            a_b = a_z
        else:
            a_a = a_c
    
    a = (a_a + a_b) / 2
    #print "a=", a
    
    # Шаг
    x[i] = x[i - 1] - a * grad(A, b, x[i - 1])
    #print x[i], func(A, b, x[i])
    
    # Результат
    x_min = x[i]
    i_min = i
    f_min = func(A, b, x_min)
    
    # Условие останова
    if np.linalg.norm(grad(A, b, x[i])) < grad_norm:
        break
    
    if i >= maxiter:
        break
        
    i += 1

np.set_printoptions(precision=3)
print "i =", i
print "i_min =", i_min
print "func =", f_min
print "x =", x_min
print "||grad|| = ||Ax-b|| =", linalg.norm(np.dot(A, x_min) - b)

i = 518
i_min = 518
func = -1.29838585245
x = [-0.025  0.06   0.084  0.05   0.011  0.097  0.033  0.09   0.077 -0.02
  0.046  0.031  0.011  0.026  0.064 -0.152  0.026  0.052  0.052  0.042
 -0.004  0.009  0.016 -0.016  0.04   0.001  0.018  0.05  -0.031  0.052
  0.051 -0.005 -0.007 -0.003  0.01   0.068  0.104  0.018  0.044  0.086
 -0.019 -0.004  0.051  0.058  0.063  0.048  0.047  0.109  0.045 -0.048
 -0.048  0.057  0.011  0.033 -0.014  0.082  0.01  -0.012 -0.002  0.048
  0.013 -0.003  0.041  0.033  0.108 -0.079 -0.045  0.     0.046  0.032
  0.098 -0.025  0.007  0.044 -0.041  0.054 -0.03   0.021  0.136  0.002
 -0.033  0.012  0.03   0.037  0.035  0.085 -0.049  0.134 -0.049 -0.044
  0.046  0.042 -0.041  0.01  -0.036  0.041  0.059  0.078  0.082  0.041
  0.002 -0.004  0.013  0.035  0.078  0.065  0.181  0.007  0.066  0.028
  0.013  0.04   0.056 -0.05  -0.107  0.011 -0.041 -0.049  0.034  0.074
  0.015  0.019 -0.025  0.022  0.004 -0.03   0.002  0.076  0.128  0.021
 -0.041  0.003 -0.081  0.04   0.

## Метод 4. Метод Ньютона
Функция квадратичная $\Rightarrow$ метод сходится за одну итерацию:
$x_{k+1}=x_k-(\nabla^2f(x_k))^{-1}\nabla f(x_k)=x_k-A^{-1}(Ax-b)=A^{-1}b$

In [191]:
i_min = 1
x_min = np.dot(np.linalg.inv(A),b)
f_min = func(A, b, x_min)

np.set_printoptions(precision=3)
print "i_min =", i_min
print "func =", f_min
print "x =", x_min
print "||grad|| = ||Ax-b|| =", linalg.norm(np.dot(A, x_min) - b)

i_min = 1
func = -1.29838585603
x = [-0.025  0.06   0.084  0.05   0.011  0.097  0.033  0.09   0.077 -0.02
  0.046  0.031  0.011  0.026  0.064 -0.152  0.026  0.052  0.052  0.042
 -0.004  0.009  0.016 -0.016  0.04   0.001  0.018  0.05  -0.031  0.052
  0.051 -0.005 -0.007 -0.003  0.01   0.068  0.104  0.018  0.044  0.086
 -0.019 -0.004  0.051  0.058  0.063  0.048  0.047  0.109  0.045 -0.048
 -0.048  0.057  0.011  0.033 -0.014  0.082  0.01  -0.012 -0.002  0.048
  0.013 -0.003  0.041  0.033  0.108 -0.079 -0.045  0.     0.046  0.032
  0.098 -0.025  0.007  0.044 -0.041  0.054 -0.03   0.021  0.136  0.002
 -0.033  0.012  0.03   0.037  0.035  0.085 -0.049  0.134 -0.049 -0.044
  0.046  0.042 -0.041  0.01  -0.036  0.041  0.059  0.078  0.082  0.041
  0.002 -0.004  0.013  0.035  0.078  0.065  0.181  0.007  0.066  0.028
  0.013  0.04   0.056 -0.05  -0.107  0.011 -0.041 -0.049  0.034  0.074
  0.015  0.019 -0.025  0.022  0.004 -0.03   0.002  0.076  0.128  0.021
 -0.041  0.003 -0.081  0.04   0.077  0.121

## Метод 5. Метод сопряженных градиентов
Для СЛАУ:
1. $x_0\in\mathbb{R}^n$
2. $s_1=-\nabla f(x_0)$
3. $\alpha_k=-\frac{\nabla^T f(x_{k-1})s_k}{s_kAs_k}$
4. $x_k=x_{k-1}+\alpha_ks_k$
5. $\beta_{k-1}=\frac{\nabla^T f(x_{k-1})As_{k-1}}{s_{k-1}^TAs_{k-1}}$
6. $s_k=-\nabla f(x_{k-1})+\beta_{k-1}s_{k-1}$

Общий случай:
1. $x_0\in\mathbb{R}^n$
2. $s_1=-\nabla f(x_0)$
3. $x_{k}=\arg\min\limits_{\alpha}f(x_{k-1}+\alpha s_k)$
4. $\beta_{k}=(\frac{||\nabla f(x_k)||}{||\nabla f(x_{k-1}||})^2$
5. $s_{k+1}=-\nabla f(x_{k})+\beta_{k} s_{k}$

In [199]:
# Массив с точками
x=list()
x.append(np.ones(n) * 9)
k = 1

# Одномерная минимизация
# Точность
# Метод хуже работает при плохой точности одномерной минимизации
a_eps = 1e-8
# Можно использовать теоретическое значение для квадратичной функции
# (строка a = ath ниже)

# Начальное значение
a0 = 1

# Условие останова
maxiter = 10000
grad_norm = 1e-6

s = -grad(A, b, x[0])

# Функция одной переменной, которую будем минимизировать
def func1(x_, alpha, s):
    return(func(A, b, x_ + alpha * s))

while True:
    x.append(0)
    
    # Выбор шага: одномерная минимизация,
    # Метод деления отрезка
    
    # Начальные границы отрезка
    a_a = 0.
    a_b = 1. * a0
    
    # Увеличиваем границу, пока функция не начнет увеличиваться
    while True:
        if func1(x[k - 1], a_b, s) > func1(x[k - 1], 0, s):
            break
        a_b *= 2
    
    #print "amax=", a_b
    
    # Ищем минимум функции одной переменной func1(x[k-1], a) по a
    while abs(a_a - a_b) >= a_eps:
        a_c = (a_a + a_b) / 2
        a_y = (a_a + a_c) / 2
        a_z = (a_y + a_b) / 2
        
        if func1(x[k - 1], a_y, s) <= func1(x[k - 1], a_c, s):
            a_b = a_c
            #a_c = a_y
        elif func1(x[k - 1], a_c, s) <= func1(x[k - 1], a_z, s):
            a_a = a_y
            a_b = a_z
        else:
            a_a = a_c
    
    a = (a_a + a_b) / 2
    #print "a=", a
    #print "s=", s
    #print "x=", x[k - 1]
    
    # Теоретическое значение a для квадратичных функций
    ath = - (np.dot(grad(A, b, x[k - 1]), s)) / (np.dot(s, np.dot(A, s)))
    #print "ath=", ath
    
    # Использовать теоретическое значение?
    # a = ath
    
    # Шаг
    x[k] = x[k - 1] + a * s
    
    # Результат
    x_min = x[k]
    f_min = func(A, b, x_min)
    
    # Новое s
    grad_norm_prev = np.linalg.norm(grad(A, b, x[k - 1]))
    grad_norm_now = np.linalg.norm(grad(A, b, x[k]))
    
    if grad_norm_now < grad_norm:
        break
    
    beta = (grad_norm_now / grad_norm_prev) ** 2
    #print "beta=", beta
    
    # Теоретическое значение  beta для квадратичных функций
    betath = (np.dot(grad(A, b, x[k]), np.dot(A, s))) / (np.dot(s, np.dot(A, s)))
    #print "betath=", betath
    #beta = betath
    
    s = -grad(A, b, x[k]) + beta * s
    
    #print "f_min=", f_min
    
    if k >= maxiter:
        break
        
    k += 1
    #break

np.set_printoptions(precision=3)
print "k =", k
print "func =", f_min
print "x =", x_min
print "||grad|| = ||Ax-b|| =", linalg.norm(np.dot(A, x_min) - b)

k = 84
func = -1.29838585603
x = [-0.025  0.06   0.084  0.05   0.011  0.097  0.033  0.09   0.077 -0.02
  0.046  0.031  0.011  0.026  0.064 -0.152  0.026  0.052  0.052  0.042
 -0.004  0.009  0.016 -0.016  0.04   0.001  0.018  0.05  -0.031  0.052
  0.051 -0.005 -0.007 -0.003  0.01   0.068  0.104  0.018  0.044  0.086
 -0.019 -0.004  0.051  0.058  0.063  0.048  0.047  0.109  0.045 -0.048
 -0.048  0.057  0.011  0.033 -0.014  0.082  0.01  -0.012 -0.002  0.048
  0.013 -0.003  0.041  0.033  0.108 -0.079 -0.045  0.     0.046  0.032
  0.098 -0.025  0.007  0.044 -0.041  0.054 -0.03   0.021  0.136  0.002
 -0.033  0.012  0.03   0.037  0.035  0.085 -0.049  0.134 -0.049 -0.044
  0.046  0.042 -0.041  0.01  -0.036  0.041  0.059  0.078  0.082  0.041
  0.002 -0.004  0.013  0.035  0.078  0.065  0.181  0.007  0.066  0.028
  0.013  0.04   0.056 -0.05  -0.107  0.011 -0.041 -0.049  0.034  0.074
  0.015  0.019 -0.025  0.022  0.004 -0.03   0.002  0.076  0.128  0.021
 -0.041  0.003 -0.081  0.04   0.077  0.121 -0

## TODO:
### Методы
* Градиентный спуск +
* Субградиентный спуск =град, т.к. функция дифференцируема
* Метод Ньютона + всего 1 шаг
* Метод сопряженных градиентов +

### Критерии остановки
не знаем $f(x^*)$ <--

### Выбор шага
По 2 методы на каждый элемент

* Армихо +
* Константа +
* Одномерная минимизация +